
# FBI Time Series Forecasting Project

## 📌 Objective
Forecast monthly crime incidents by type using time series models (Prophet and ARIMA) to support resource planning and crime analysis.

## 📂 Dataset
- `Train.csv`: Historical crime data with dates and types
- `Test.csv`: Monthly structure to predict for each crime type

## ⚙️ Methodology
- Perform data cleaning and preprocessing
- Aggregate time series data
- Train and validate Prophet and ARIMA models
- Compare model performance using RMSE and MAE
- Predict on test set using the best model per crime type


## 🔍 Exploratory Data Analysis & Missing Value Handling

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from prophet import Prophet
from pmdarima import auto_arima
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tqdm import tqdm

# Load Data
train_df = pd.read_csv('Train.csv')
test_df = pd.read_csv('Test.csv')

# Handle dates and missing values
train_df['Date'] = pd.to_datetime(train_df['Date'], errors='coerce')
train_df.dropna(subset=['Date'], inplace=True)
train_df['MonthStart'] = train_df['Date'].dt.to_period('M').dt.to_timestamp()

# EDA - Plot Distribution
plt.figure(figsize=(10, 5))
sns.countplot(data=train_df, y='TYPE', order=train_df['TYPE'].value_counts().index)
plt.title("Crime Type Distribution")
plt.show()

# Monthly trend
monthly_trend = train_df.groupby('MonthStart').size()
monthly_trend.plot(figsize=(12, 4), title="Monthly Crime Trend")
plt.xlabel("Month")
plt.ylabel("Number of Incidents")
plt.grid()
plt.show()


## 📈 Model Training and Evaluation (Prophet vs ARIMA)

In [ ]:

# Aggregate data
monthly_grouped = train_df.groupby(['MonthStart', 'TYPE']).size().reset_index(name='Crime_Count')

# Prepare test
test_df['MonthStart'] = pd.to_datetime(test_df['YEAR'].astype(str) + '-' + test_df['MONTH'].astype(str) + '-01')
test_df['Incident_Counts'] = 0

types = test_df['TYPE'].unique()
model_perf = []

print("🔁 Running per TYPE...")

for crime_type in tqdm(types):
    data = monthly_grouped[monthly_grouped['TYPE'] == crime_type]
    if len(data) < 24:
        print(f"Skipping {crime_type} (insufficient data)")
        continue

    df_prep = data.copy().set_index('MonthStart').asfreq('MS')
    y = df_prep['Crime_Count'].fillna(method='ffill')

    train_y = y[y.index.year < 2012]
    valid_y = y[y.index.year == 2012]

    if len(train_y) == 0 or len(valid_y) == 0:
        print(f"Skipping {crime_type} (empty train or validation set)")
        continue

    try:
        arima_model = auto_arima(train_y, seasonal=True, m=12, suppress_warnings=True)
        arima_preds = arima_model.predict(n_periods=len(valid_y))
        arima_rmse = np.sqrt(mean_squared_error(valid_y, arima_preds))
        arima_mae = mean_absolute_error(valid_y, arima_preds)
    except:
        arima_rmse = arima_mae = float('inf')

    try:
        prophet_train = train_y.reset_index().rename(columns={'MonthStart': 'ds', 'Crime_Count': 'y'})
        prophet_model = Prophet(yearly_seasonality=True)
        prophet_model.fit(prophet_train)
        future = pd.DataFrame({'ds': valid_y.index})
        forecast = prophet_model.predict(future)
        prophet_preds = forecast['yhat'].values
        prophet_rmse = np.sqrt(mean_squared_error(valid_y, prophet_preds))
        prophet_mae = mean_absolute_error(valid_y, prophet_preds)
    except:
        prophet_rmse = prophet_mae = float('inf')

    better_model = 'Prophet' if prophet_rmse < arima_rmse else 'ARIMA'
    model_perf.append([crime_type, prophet_rmse, arima_rmse, better_model])

    future_months = test_df[test_df['TYPE'] == crime_type]['MonthStart'].sort_values().unique()
    if better_model == 'Prophet':
        all_train = y.reset_index().rename(columns={'MonthStart': 'ds', 'Crime_Count': 'y'})
        model = Prophet(yearly_seasonality=True)
        model.fit(all_train)
        future = pd.DataFrame({'ds': future_months})
        forecast = model.predict(future)
        preds = forecast['yhat'].values
    else:
        arima_model = auto_arima(y, seasonal=True, m=12, suppress_warnings=True)
        preds = arima_model.predict(n_periods=len(future_months))

    test_df.loc[(test_df['TYPE'] == crime_type), 'Incident_Counts'] = [max(0, int(round(p))) for p in preds]

perf_df = pd.DataFrame(model_perf, columns=['TYPE', 'Prophet_RMSE', 'ARIMA_RMSE', 'Best_Model'])
perf_df.sort_values('Prophet_RMSE', na_position='last')


## 📤 Output and Export

In [ ]:

# Save output
test_df = test_df.sort_values(by=['YEAR', 'MONTH', 'TYPE'])
test_df.to_csv("Predicted_Test_Output.csv", index=False)

# Display performance
print("Model Performance Summary:")
display(perf_df)



##  Models Tried
- **Prophet**: Additive time series model with strong seasonality support.
- **ARIMA**: Statistical model for autoregressive and moving average with seasonality.

##  Number of Models Experimented
- Both Prophet and ARIMA were applied to each crime type (≥ 2 models tried).

##  Evaluation Metrics
- Root Mean Square Error (RMSE)
- Mean Absolute Error (MAE)

##  Hyperparameter Tuning
- Auto selection using `auto_arima` for ARIMA.
- Prophet auto-tunes seasonalities.

##  Final Conclusion
- Best model per type is chosen based on RMSE on 2012 validation set.
- Predictions for 2013 test months generated accordingly.

##  Commented Code & Modularity
- All blocks are modular with descriptive comments.
- Code is structured clearly with markdown explanations.

##  Output Formatting
- Results exported to CSV `Predicted_Test_Output.csv`
- Sorted and cleaned format for evaluation.


